In [ ]:
# Copyright 2025 Sony Corporation

## Plot for Each Gate's F

In [78]:
import subprocess
import os
import glob
import shutil
import sys
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import copy

In [79]:
workdir=os.getcwd()

In [80]:
#samples
samplenames = ["FP7000_34c","Levine_32dim","Levine_13dim","Samusik_01","Samusik_all"]


In [81]:
#here write a list of each gate's cell num
Gate_Cell_num_dict = {"Levine_32dim":[1207,3905,2248,3295,304,916,26366,20108,16520,21099,1238,330,6135,513],
                 "Levine_13dim":[912,6779,1278,253,12030,73,261,502,7796,608,13964,7821,3684,194,152,3025,6987,9564,3864,468,293,5,240,994],
                 "Samusik_01":[266,292,406,798,13607,59,197,5173,35,703,3,3508,8768,1123,12045,199,116,420,197,189,961,1016,2974,118],
                 "Samusik_all":[4241,4067,5646,12479,99356,480,2041,53495,387,6068,29,49717,114412,21137,82384,2980,1084,5469,1229,3324,8479,7855,26421,1606],
                 "FP7000_34c":[163434,349994,243746,5375,152047,95504,162707,385031,20116,20221,242194,12520,27591,48556],
                }
Gate_name_dict = {"Levine_32dim":["Basophils","CD16-_NK_cells","CD16+_NK_cells","CD34+CD38+CD123-_HSPCs","CD34+CD38+CD123+_HSPCs","CD34+CD38lo_HSCs","CD4_T_cells","CD8_T_cells","Mature_B_cells","Monocytes","pDCs","Plasma_B_cells","Pre_B_cells","Pro_B_cells"],
                 "Levine_13dim":["CD11b-_Monocyte_cells","CD11bhi_Monocyte_cells","CD11bmid_Monocyte_cells","CMP_cells","Erythroblast_cells","GMP_cells","HSC_cells","Immature_B_cells","Mature_CD38lo_B_cells","Mature_CD38mid_B_cells","Mature_CD4+_T_cells","Mature_CD8+_T_cells","Megakaryocyte_cells","MEP_cells","MPP_cells","Myelocyte_cells","Naive_CD4+_T_cells","Naive_CD8+_T_cells","NK_cells","Plasma_cell_cells","Plasmacytoid_DC_cells","Platelet_cells","Pre-B_I_cells","Pre-B_II_cells"],
                 "Samusik_01": ["B-cell Frac A-C (pro-B cells)","Basophils","CD4 T cells","CD8 T cells","Classical Monocytes","CLP","CMP","Eosinophils","gd T cells","GMP","HSC","IgD- IgMpos B cells","IgDpos IgMpos B cells","IgM- IgD- B-cells","Intermediate Monocytes","Macrophages","mDCs","MEP","MPP","NK cells","NKT cells","Non-Classical Monocytes","pDCs","Plasma Cells"],
                 "Samusik_all":["B-cell Frac A-C (pro-B cells)","Basophils","CD4 T cells","CD8 T cells","Classical Monocytes","CLP","CMP","Eosinophils","gd T cells","GMP","HSC","IgD- IgMpos B cells","IgDpos IgMpos B cells","IgM- IgD- B-cells","Intermediate Monocytes","Macrophages","mDCs","MEP","MPP","NK cells","NKT cells","Non-Classical Monocytes","pDCs","Plasma Cells"],
                  "FP7000_34c":["Naive CD4+","CM CD4+","EM CD4+","EMRA CD4+","Naive CD8+","CM CD8+","EM CD8+","EMRA CD8+","Treg","TCRγδ+","CD27- IgD+","CD27+ IgD+","CD27+ IgD-","CD27- IgD-"],
                 }

In [82]:
fsom_experiment_name = "Flowsom"
BL_experiment_name = "BL_Flowsom"
meta_list = range(10,61,1)

In [90]:
def gate_name_with_samplenum(gatenames,cell_nums):
    ret_gatenames = []
    i = 0
    for gate in gatenames:
        gatename = gate + " (" + "%s"%cell_nums[i] + ")"
        ret_gatenames.append(gatename)
        i = i + 1
    return ret_gatenames

In [ ]:
for samplename in samplenames:
    iteration = 10
    s_df = pd.DataFrame()
    gate_cell_num = Gate_Cell_num_dict.get(samplename)
    gatename = Gate_name_dict.get(samplename)
    gate_num = len(gatename)
    for meta in meta_list:
        #if meta < gate_num:
        #    continue
        dir_path = os.path.join(workdir, fsom_experiment_name, samplename, "%s"%meta, "%s"%iteration)
        csv_files =  [os.path.join(dir_path,file) for file in os.listdir(dir_path) if file.endswith(".csv")]
        c_df = pd.DataFrame()
        data_list = []
        for file in csv_files:
            data= pd.read_csv(file,header=None)
            data_list.append(data)
        c_df = pd.concat(data_list, axis=1)
        ave = c_df.mean(axis=1)
        row_means_df = pd.DataFrame(ave,columns=["%s"%meta])
        s_df = pd.concat([s_df, row_means_df],axis=1)
 
    gatename = Gate_name_dict.get(samplename)
    gatenames = gate_name_with_samplenum(gatename, gate_cell_num)
    save_df = copy.deepcopy(s_df)
    
    s_df.index = gatenames
    s_df["cell_num"] = gate_cell_num
    ss_df  = s_df.sort_values(by="cell_num",ascending=False).drop("cell_num",axis=1)
    #ss_df.loc["ave"] = ss_df.mean(axis=0)
    fig, ax = plt.subplots(figsize = (10,10))
    ax.set_title(samplename)
    sns.heatmap(ss_df, cmap="RdYlBu_r", ax = ax,vmax=1.0, vmin=0.0)
    plt.xticks(range(0, len(ss_df.columns), 5), ss_df.columns[::5])
    out_fname = "fsom_meta_vs_each_f_%s.png"%samplename
    plt.savefig(out_fname, dpi=300,bbox_inches="tight")
    filename = "%s_fsom_meta_vs_each_f.csv"%samplename
    ss_df.to_csv(filename)

In [ ]:
for samplename in samplenames:
    iteration = 10
    s_df = pd.DataFrame()
    gatename = Gate_name_dict.get(samplename)
    gate_cell_num = Gate_Cell_num_dict.get(samplename)
    gate_num = len(gatename)
    for meta in meta_list:
        #if meta < gate_num:
        #    continue
        dir_path = os.path.join(workdir, BL_experiment_name, samplename)
        f = "f1_meta_%s_it_%s_hang.hres_each.csv"%(meta,iteration)
        file =os.path.join(dir_path, f) 
        data= pd.read_csv(file,header=None)
        row_means_df = data.rename(columns={0:"%s"%meta})
        s_df = pd.concat([s_df, row_means_df],axis=1)
    
    gatename = Gate_name_dict.get(samplename)
    gatenames = gate_name_with_samplenum(gatename, gate_cell_num)
    save_df = copy.deepcopy(s_df)

    s_df.index = gatenames
    gatename = Gate_name_dict.get(samplename)
    s_df["cell_num"] = gate_cell_num
    ss_df  = s_df.sort_values(by="cell_num",ascending=False).drop("cell_num",axis=1)
    #ss_df.loc["ave"] = ss_df.mean(axis=0)

    fig, ax = plt.subplots(figsize = (10,10))
    ax.set_title(samplename)
    sns.heatmap(ss_df, cmap="RdYlBu_r", ax = ax, vmax=1.0, vmin=0.0)
    plt.xticks(range(0, len(ss_df.columns), 5), ss_df.columns[::5])
    out_fname = "BLfsom_meta_vs_each_f_%s.png"%samplename
    plt.savefig(out_fname, dpi=300,bbox_inches="tight")
    filename = "%s_BLsom_meta_vs_each_f.csv"%samplename
    ss_df.to_csv(filename)

# plot best meta at each gate (flowsom) vs best meta at each gate (BL-FlowSOM) 

In [ ]:
for samplename in samplenames:
    iteration = 10
    s_df = pd.DataFrame()
    for meta in meta_list:
        dir_path = os.path.join(workdir, fsom_experiment_name, samplename, "%s"%meta, "%s"%iteration)
        csv_files =  [os.path.join(dir_path,file) for file in os.listdir(dir_path) if file.endswith(".csv")]
        c_df = pd.DataFrame()
        data_list = []
        for file in csv_files:
            data= pd.read_csv(file,header=None)
            seed = int(os.path.basename(file).split("_")[2])
            data.columns = ["%s"%seed]
            data_list.append(data)
        c_df = pd.concat(data_list, axis=1)
        ave = c_df.mean(axis=1)
        row_means_df = pd.DataFrame(ave,columns=["%s"%meta])
        s_df = pd.concat([s_df, row_means_df],axis=1)

    
    #max_idx(meta div) of each gate
    max_metas = s_df.idxmax(axis=1)
    gate_id = 0
    gate_df = pd.DataFrame()
    for meta in max_metas:
        dir_path = os.path.join(workdir, fsom_experiment_name, samplename, "%s"%meta, "%s"%iteration)
        csv_files =  [os.path.join(dir_path,file) for file in os.listdir(dir_path) if file.endswith(".csv")]
        c_df = pd.DataFrame()
        data_list = []
        for file in csv_files:
            data= pd.read_csv(file,header=None)
            seed = int(os.path.basename(file).split("_")[2])
            data.columns = ["%s"%seed]
            data_list.append(data)
        c_df = pd.concat(data_list, axis=1)
        d = pd.DataFrame(c_df.loc[gate_id])
        d.columns = ["%d"%gate_id]
        gate_df = pd.concat([gate_df, d],axis=1) 
        gate_id = gate_id + 1
    gatename = Gate_name_dict.get(samplename)
    gate_df.columns = gatename
    gs = Gate_Cell_num_dict.get(samplename)
    c_df = pd.DataFrame(gs)
    c_df.columns = ["cell_num"]
    c_df.index = gatename
    c_df_t = c_df.T
    gate_df = pd.concat([gate_df, c_df_t],axis=0)
    
    
    gatenames = gate_name_with_samplenum(gatename, gs)
    gate_df.columns = gatenames
    gate_df = gate_df.sort_values(by="cell_num",axis=1, ascending=False).drop(["cell_num"])


    
    fig = plt.figure(figsize=(8,4),dpi=300)
    sns.set(style="whitegrid")
    sns.boxplot(showmeans=False,data = gate_df, color="b",width=0.3)

    #BL data
    f = "%s_BLsom_meta_vs_each_f.csv"%samplename
    fval_df = pd.read_csv(f,index_col=0)
    fval_dict = fval_df.max(axis=1)

    i = 0
    for gate in gate_df.columns:
        F = fval_dict.get(gate)
        plt.plot(i, F , marker='o', markersize=2, color='red', label="a",zorder=10)  # average       
        i = i+1
    plt.suptitle("%s"%samplename)
    
    plt.xticks(rotation=90)
    fout = "gate_max_F_%s.png"%samplename
    fig.savefig(fout,bbox_inches="tight")


# plot best meta div(flowsom) vs best meta(BL-FlowSOM) 

In [ ]:
for samplename in samplenames:
    iteration = 10
    s_df = pd.DataFrame()
    
    meta_f = pd.read_csv("%s_fsom_meta_vs_each_f.csv"%samplename,index_col=0)
    maxfsom_meta = int(meta_f.mean().idxmax())
    gatename = Gate_name_dict.get(samplename)
    
    gatenum = len(gatename)
    
    gate_df = pd.DataFrame()
    for gate_id  in range(0,gatenum):
        dir_path = os.path.join(workdir, fsom_experiment_name, samplename, "%s"%maxfsom_meta, "%s"%iteration)
        csv_files =  [os.path.join(dir_path,file) for file in os.listdir(dir_path) if file.endswith(".csv")]
        c_df = pd.DataFrame()
        data_list = []
        for file in csv_files:
            data= pd.read_csv(file,header=None)
            seed = int(os.path.basename(file).split("_")[2])
            data.columns = ["%s"%seed]
            data_list.append(data)
        c_df = pd.concat(data_list, axis=1)
        d = pd.DataFrame(c_df.loc[gate_id])
        d.columns = ["%d"%gate_id]
        gate_df = pd.concat([gate_df, d],axis=1) 

    gatename = Gate_name_dict.get(samplename)
    gate_df.columns = gatename
    gs = Gate_Cell_num_dict.get(samplename)
    c_df = pd.DataFrame(gs)
    c_df.columns = ["cell_num"]
    c_df.index = gatename
    c_df_t = c_df.T
    gate_df = pd.concat([gate_df, c_df_t],axis=0)
    
    
    gatenames = gate_name_with_samplenum(gatename, gs)
    gate_df.columns = gatenames
    gate_df = gate_df.sort_values(by="cell_num",axis=1, ascending=False).drop(["cell_num"])

    fig = plt.figure(figsize=(8,4),dpi=300)
    sns.set(style="whitegrid")
    sns.boxplot(showmeans=False,data = gate_df, color="b",width=0.3)

    #BL data
    f = "%s_BLsom_meta_vs_each_f.csv"%samplename
    fval_df = pd.read_csv(f,index_col=0)
    max_bl_div = fval_df.mean().idxmax()
    
    i = 0
    for gate in gate_df.columns:
        F = fval_df.loc[gate].get(max_bl_div)
        plt.plot(i, F , marker='o', markersize=2, color='red', label="a",zorder=10)  # average       
        i = i+1
    plt.suptitle("%s"%samplename)
    
    plt.xticks(rotation=90)
    fout = "gate_max_F_best_meta_fixed_%s.png"%samplename
    fig.savefig(fout,bbox_inches="tight")
